In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import historical weather data

dfFull = pd.read_csv('data-raw/BZN-weather-data_4.1895-6.2015.csv',
                        na_values=['-9999','9999'],
                        index_col='DATE',
                        parse_dates=True)

/Users/ericdietrich/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (2,3,4,39,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Select columns of interest from full data set
df = dfFull.loc[:,['PRCP','TMAX','TMIN']]
df['day'] = df.index.day
df['month'] = df.index.month

# Calculate record high and add to output_df
record_max = df[['TMAX','month','day']].groupby(['month','day']).aggregate([max, np.mean])
record_max.columns = ['RECHIGH', 'AVGHIGH']
df = pd.merge(df, record_max, left_on=['month','day'], right_index=True, how='left').sort_index()

# Calculate record low and add to output_df
record_min = df[['TMIN','month','day']].groupby(['month','day']).aggregate([min, np.mean])
record_min.columns = ['RECLOW', 'AVGLOW']
df = pd.merge(df, record_min, left_on=['month','day'], right_index=True, how='left').sort_index()

df.head()

,PRCP,TMAX,TMIN,day,month,RECHIGH,AVGHIGH,RECLOW,AVGLOW
DATE,,,,,,,,,
1892-04-08 00:00:00,0,78,-150,8,4,239,109.105691,-172,-22.284553
1892-04-09 00:00:00,0,128,6,9,4,250,111.983740,-128,-17.008130
1892-04-10 00:00:00,0,100,6,10,4,244,107.577236,-150,-15.528455
1892-04-11 00:00:00,0,89,-6,11,4,239,113.447154,-117,-13.252033
1892-04-12 00:00:00,0,94,-28,12,4,244,121.804878,-156,-15.674797


In [4]:
# Read in snowpack data

# Snow water equivalent data since ~1994, snow depth data since 6-20-2002
# Snow water equivalent defined here:
# http://www.nrcs.usda.gov/wps/portal/nrcs/detail/or/snow/?cid=nrcs142p2_046155

dfSnow = pd.read_csv('data-raw/bracket-creek_11-22-15.csv', index_col='Date',
                        parse_dates=True, skiprows=7)
dfSnow.rename(columns={'Snow Depth (in)':'SNWDEP'}, inplace=True)

# Select only SNWDEP column, remove nulls
dfSnow = pd.DataFrame(dfSnow['SNWDEP'])
dfSnow = dfSnow[dfSnow['SNWDEP'].notnull()]

In [5]:
# Strip df to date range of interest
# Data currently runs '1892-04-08' to '2015-06-30'

# Export dates
start_date = pd.to_datetime('2013-08-01')
end_date = pd.to_datetime('2014-07-31')

dfOut = df.loc[start_date:end_date]

In [6]:
# Add SNWDEP column to dfOut
dfOut = pd.merge(dfOut, dfSnow, right_index=True, left_index=True, how='left').sort_index()

In [7]:
dfOut.head()

,PRCP,TMAX,TMIN,day,month,RECHIGH,AVGHIGH,RECLOW,AVGLOW,SNWDEP
DATE,,,,,,,,,,
2013-08-01,3,283,133,1,8,361,280.058824,39,111.924370,0
2013-08-02,43,250,106,2,8,344,275.857143,28,106.949580,0
2013-08-03,0,239,111,3,8,361,277.739496,33,109.084034,0
2013-08-04,0,261,111,4,8,367,274.873950,50,103.773109,0
2013-08-05,0,278,122,5,8,372,277.016807,39,105.453782,0


In [8]:
filename = "BZN-weather_" + str(start_date.date()) + "_" + str(end_date.date()) + ".csv"
dfOut.to_csv('data-processed/' + filename)
print 'Output file: ' + filename

Output file: BZN-weather_2013-08-01_2014-07-31.csv
